In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Coronary_artery_disease"
cohort = "GSE64554"

# Input paths
in_trait_dir = "../../input/GEO/Coronary_artery_disease"
in_cohort_dir = "../../input/GEO/Coronary_artery_disease/GSE64554"

# Output paths
out_data_file = "../../output/preprocess/Coronary_artery_disease/GSE64554.csv"
out_gene_data_file = "../../output/preprocess/Coronary_artery_disease/gene_data/GSE64554.csv"
out_clinical_data_file = "../../output/preprocess/Coronary_artery_disease/clinical_data/GSE64554.csv"
json_path = "../../output/preprocess/Coronary_artery_disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"GE/miRNA expression profile of Human Epicardial Adipose Tissue (EAT) and Subcutaneous Adipose Tissue (SAT) in Patients with Coronary Artery Disease (CAD) vs. Controls (CTRL) - PART 1 - Genes"
!Series_summary	"Gene expression profiles of Human EAT vs. SAT (CTRL & CAD). The aim of the present study was to assess a gene expression chart characterizing EAT vs. SAT, and CAD vs. CTRL. Results provide the information that EAT is characterized by a differential expression of different genes when compared to its reference tissue (SAT), and that EAT is characterized by specific gene expression changes in patients with CAD."
!Series_overall_design	"RNA obtained from EAT & SAT of the same patients (paired samples). Comparisons: EAT vs. SAT (paired samples) & CAD vs. CTRL"
Sample Characteristics Dictionary:
{0: ['age: 76', 'age: 71', 'age: 57', 'age: 30', 'age: 81', 'age: 51', 'age: 36', 'age: 44', 'age: 52', 'age: 73', 'age: 69', 'age: 56', 'age: 70', 'age: 6

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on Series_title, this is gene expression data from adipose tissue
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (coronary artery disease status)
# Key 2 contains "disease state: control" and "disease state: coronary artery disease"
trait_row = 2

# For age - available in key 0
age_row = 0

# For gender - not available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion

def convert_trait(value):
    """Convert disease state to binary: 0 for control, 1 for CAD"""
    if not isinstance(value, str):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = value.lower().strip()
    
    if 'control' in value:
        return 0
    elif 'coronary artery disease' in value or 'cad' in value:
        return 1
    else:
        return None

def convert_age(value):
    """Convert age to continuous numeric value"""
    if not isinstance(value, str):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender to binary: 0 for female, 1 for male
    Not used in this dataset as gender information is not available"""
    return None

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    clinical_data_processed = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the processed clinical data
    preview = preview_df(clinical_data_processed)
    print("Preview of processed clinical data:")
    print(preview)
    
    # Save the processed clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_data_processed.to_csv(out_clinical_data_file)
    print(f"Processed clinical data saved to {out_clinical_data_file}")


Preview of processed clinical data:
{'GSM1574149': [0.0, 76.0], 'GSM1574150': [0.0, 76.0], 'GSM1574151': [0.0, 71.0], 'GSM1574152': [0.0, 71.0], 'GSM1574153': [0.0, 57.0], 'GSM1574154': [0.0, 57.0], 'GSM1574155': [0.0, 30.0], 'GSM1574156': [0.0, 30.0], 'GSM1574157': [0.0, 81.0], 'GSM1574158': [0.0, 81.0], 'GSM1574159': [0.0, 51.0], 'GSM1574160': [0.0, 51.0], 'GSM1574161': [0.0, 36.0], 'GSM1574162': [0.0, 36.0], 'GSM1574163': [0.0, 44.0], 'GSM1574164': [0.0, 44.0], 'GSM1574165': [0.0, 36.0], 'GSM1574166': [0.0, 36.0], 'GSM1574167': [0.0, 52.0], 'GSM1574168': [0.0, 52.0], 'GSM1574169': [1.0, 73.0], 'GSM1574170': [1.0, 73.0], 'GSM1574171': [1.0, 69.0], 'GSM1574172': [1.0, 69.0], 'GSM1574173': [1.0, 56.0], 'GSM1574174': [1.0, 56.0], 'GSM1574175': [1.0, 69.0], 'GSM1574176': [1.0, 69.0], 'GSM1574177': [1.0, 73.0], 'GSM1574178': [1.0, 73.0], 'GSM1574179': [1.0, 70.0], 'GSM1574180': [1.0, 70.0], 'GSM1574181': [1.0, 67.0], 'GSM1574182': [1.0, 67.0], 'GSM1574183': [1.0, 60.0], 'GSM1574184': [1.0

### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
marker_row = None
try:
    with gzip.open(matrix_file, 'rt') as file:
        for i, line in enumerate(file):
            if "!series_matrix_table_begin" in line:
                found_marker = True
                marker_row = i
                print(f"Found the matrix table marker at line {i}")
                break
    
    if not found_marker:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        is_gene_available = False
        
    # If marker was found, try to extract gene data
    if is_gene_available:
        try:
            # Try using the library function
            gene_data = get_genetic_data(matrix_file)
            
            if gene_data.shape[0] == 0:
                print("Warning: Extracted gene data has 0 rows.")
                is_gene_available = False
            else:
                print(f"Gene data shape: {gene_data.shape}")
                # Print the first 20 gene/probe identifiers
                print("First 20 gene/probe identifiers:")
                print(gene_data.index[:20].tolist())
        except Exception as e:
            print(f"Error extracting gene data with get_genetic_data(): {e}")
            is_gene_available = False
    
    # If gene data extraction failed, examine file content to diagnose
    if not is_gene_available:
        print("Examining file content to diagnose the issue:")
        try:
            with gzip.open(matrix_file, 'rt') as file:
                # Print lines around the marker if found
                if marker_row is not None:
                    for i, line in enumerate(file):
                        if i >= marker_row - 2 and i <= marker_row + 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        if i > marker_row + 10:
                            break
                else:
                    # If marker not found, print first 10 lines
                    for i, line in enumerate(file):
                        if i < 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        else:
                            break
        except Exception as e2:
            print(f"Error examining file: {e2}")
        
except Exception as e:
    print(f"Error processing file: {e}")
    is_gene_available = False

# Update validation information if gene data extraction failed
if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")
    # Update the validation record since gene data isn't available
    is_trait_available = False  # We already determined trait data isn't available in step 2
    validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path,
                                 is_gene_available=is_gene_available, is_trait_available=is_trait_available)


SOFT file: ../../input/GEO/Coronary_artery_disease/GSE64554/GSE64554_family.soft.gz
Matrix file: ../../input/GEO/Coronary_artery_disease/GSE64554/GSE64554_series_matrix.txt.gz
Found the matrix table marker at line 56


Gene data shape: (48783, 46)
First 20 gene/probe identifiers:
['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209', 'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229', 'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236', 'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253', 'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262']


### Step 4: Gene Identifier Review

In [5]:
# Analysis of gene identifiers
# The identifiers starting with "ILMN_" are Illumina probe IDs, not human gene symbols
# These are microarray probe identifiers from Illumina platform that need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=3))

# Looking at the output, the Symbol column seems to contain gene information
print("\nExamining ID and Symbol columns format (first 3 rows):")
if 'ID' in gene_annotation.columns and 'Symbol' in gene_annotation.columns:
    for i in range(min(3, len(gene_annotation))):
        print(f"Row {i}: ID={gene_annotation['ID'].iloc[i]}")
        print(f"Symbol: {gene_annotation['Symbol'].iloc[i]}")

    # Check the quality and completeness of the mapping
    non_null_symbols = gene_annotation['Symbol'].notna().sum()
    total_rows = len(gene_annotation)
    print(f"\nSymbol column completeness: {non_null_symbols}/{total_rows} rows ({non_null_symbols/total_rows:.2%})")
    
    # Check if some extracted gene symbols can be found in the Symbol column
    print("\nAttempting to extract gene symbols from the first few rows:")
    for i in range(min(3, len(gene_annotation))):
        if pd.notna(gene_annotation['Symbol'].iloc[i]):
            symbols = extract_human_gene_symbols(str(gene_annotation['Symbol'].iloc[i]))
            print(f"Row {i} extracted symbols: {symbols}")



Gene annotation preview:
Columns in gene annotation: ['ID', 'nuID', 'Species', 'Source', 'Search_Key', 'Transcript', 'ILMN_Gene', 'Source_Reference_ID', 'RefSeq_ID', 'Unigene_ID', 'Entrez_Gene_ID', 'GI', 'Accession', 'Symbol', 'Protein_Product', 'Array_Address_Id', 'Probe_Type', 'Probe_Start', 'SEQUENCE', 'Chromosome', 'Probe_Chr_Orientation', 'Probe_Coordinates', 'Cytoband', 'Definition', 'Ontology_Component', 'Ontology_Process', 'Ontology_Function', 'Synonyms', 'Obsolete_Probe_Id', 'GB_ACC']
{'ID': ['ILMN_1725881', 'ILMN_1910180', 'ILMN_1804174'], 'nuID': ['rp13_p1x6D80lNLk3c', 'NEX0oqCV8.er4HVfU4', 'KyqQynMZxJcruyylEU'], 'Species': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Source': ['RefSeq', 'Unigene', 'RefSeq'], 'Search_Key': ['ILMN_44919', 'ILMN_127219', 'ILMN_139282'], 'Transcript': ['ILMN_44919', 'ILMN_127219', 'ILMN_139282'], 'ILMN_Gene': ['LOC23117', 'HS.575038', 'FCGR2B'], 'Source_Reference_ID': ['XM_933824.1', 'Hs.575038', 'XM_938851.1'], 'RefSeq_ID': ['XM_933824.

### Step 6: Gene Identifier Mapping

In [7]:
# Based on the observation from Step 5, the ID column in gene_annotation contains Illumina probe IDs
# that match the gene expression data identifiers, and the Symbol column contains gene symbols

# 1 & 2. Extract gene identifier and gene symbol columns for mapping
prob_col = 'ID'
gene_col = 'Symbol'
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)

print(f"Gene mapping shape: {gene_mapping.shape}")
print("Sample of gene mapping (first 5 rows):")
print(gene_mapping.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression
gene_data = apply_gene_mapping(gene_data, gene_mapping)

print(f"Mapped gene data shape: {gene_data.shape}")
print("First few gene symbols after mapping:")
print(gene_data.index[:10].tolist())

# Normalize gene symbols to handle synonyms and variants
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data.shape}")
print("First few normalized gene symbols:")
print(gene_data.index[:10].tolist())

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping shape: (36157, 2)
Sample of gene mapping (first 5 rows):
             ID       Gene
0  ILMN_1725881   LOC23117
2  ILMN_1804174     FCGR2B
3  ILMN_1796063     TRIM44
4  ILMN_1811966  LOC653895
5  ILMN_1668162    DGAT2L3
Mapped gene data shape: (19113, 46)
First few gene symbols after mapping:
['A1BG', 'A1CF', 'A26A1', 'A26B1', 'A26C1B', 'A26C3', 'A2BP1', 'A2M', 'A2ML1', 'A3GALT2']


Gene data shape after normalization: (18319, 46)
First few normalized gene symbols:
['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS', 'AACS']


Gene expression data saved to ../../output/preprocess/Coronary_artery_disease/gene_data/GSE64554.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Attempt to load gene data and handle possible issues with normalization
try:
    # Create output directory if it doesn't exist
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    
    # Check if gene_data (from previous step) has any content
    if gene_data.shape[0] == 0:
        print("WARNING: Gene data is empty after normalization in previous step.")
        print("This appears to be miRNA data rather than gene expression data.")
        
        # Since gene_data is empty, set gene_available to False
        is_gene_available = False
        
        # Create an empty dataframe for metadata purposes
        empty_df = pd.DataFrame()
        
        # Log information about this dataset for future reference
        validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available,
            is_biased=True,  # Consider it biased as we can't use it
            df=empty_df,
            note="Dataset appears to contain miRNA data rather than gene expression data. Gene symbols could not be normalized."
        )
        
        print("Dataset marked as unusable due to lack of valid gene expression data.")
    else:
        # If gene_data is not empty, proceed with normalization and linking
        print(f"Gene data shape before normalization: {gene_data.shape}")
        
        # Save the gene data we have, even if it's already normalized
        gene_data.to_csv(out_gene_data_file)
        print(f"Gene data saved to {out_gene_data_file}")
        
        # Attempt to link clinical and gene data
        if is_trait_available:
            # Load clinical data
            clinical_features = pd.read_csv(out_clinical_data_file, index_col=0)
            print(f"Loaded clinical data shape: {clinical_features.shape}")
            
            # Link the clinical and genetic data
            linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
            print(f"Initial linked data shape: {linked_data.shape}")
            
            # Handle missing values
            linked_data = handle_missing_values(linked_data, trait)
            print(f"Linked data shape after handling missing values: {linked_data.shape}")
            
            if linked_data.shape[0] > 0:
                # Check for bias in trait and demographic features
                is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
                
                # Validate data quality and save cohort info
                is_usable = validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=is_biased,
                    df=linked_data,
                    note="Successfully processed gene expression data for coronary artery disease."
                )
                
                # Save the linked data if it's usable
                if is_usable:
                    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                    linked_data.to_csv(out_data_file)
                    print(f"Linked data saved to {out_data_file}")
                else:
                    print("Data not usable for trait study - not saving final linked data.")
            else:
                print("After handling missing values, no samples remain.")
                validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=True,
                    df=pd.DataFrame(),
                    note="No valid samples after handling missing values."
                )
        else:
            # Cannot proceed with linking if trait data is missing
            validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=is_gene_available,
                is_trait_available=is_trait_available,
                is_biased=True,
                df=pd.DataFrame(),
                note="Cannot link data because trait information is not available."
            )
except Exception as e:
    print(f"Error in data processing: {e}")
    
    # Log the error and mark the dataset as unusable
    validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=False,  # Consider gene data unavailable if we had an error
        is_trait_available=is_trait_available,
        is_biased=True,  # Consider it biased as we can't use it
        df=pd.DataFrame(),  # Empty dataframe for metadata
        note=f"Error during normalization or linking: {str(e)}"
    )

Gene data shape before normalization: (18319, 46)


Gene data saved to ../../output/preprocess/Coronary_artery_disease/gene_data/GSE64554.csv
Loaded clinical data shape: (2, 46)
Initial linked data shape: (46, 18321)


Linked data shape after handling missing values: (46, 18321)
For the feature 'Coronary_artery_disease', the least common label is '0.0' with 20 occurrences. This represents 43.48% of the dataset.
The distribution of the feature 'Coronary_artery_disease' in this dataset is fine.

Quartiles for 'Age':
  25%: 53.0
  50% (Median): 69.0
  75%: 72.5
Min: 30.0
Max: 86.0
The distribution of the feature 'Age' in this dataset is fine.



Linked data saved to ../../output/preprocess/Coronary_artery_disease/GSE64554.csv
